# CDS API - Atmosphere Copernicus Request

GitHub Repository containing examples from ECMWF: https://github.com/ecmwf-projects/copernicus-training

Change key/value provided in the .cdsapirc file as explained in the following links:
- Climate -> ERA5 https://cds.climate.copernicus.eu/api-how-to
- Atmosphere -> CAMS https://ads.atmosphere.copernicus.eu/api-how-to 

Or use the Key/Value:

- url: https://ads.atmosphere.copernicus.eu/api/v2
- key: 7829:e0d40eca-947c-4eb8-91f5-c5cfbcaa2172
<br><br>
- url: https://cds.climate.copernicus.eu/api/v2
- key: 114120:ff103cbe-66d7-4fde-bc68-0bb30039dc50



Link to Copernicus Services:
- Link to ADS Atmosphere Copernicus: https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview
- Link to CDS Climate Copernicus: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview



In [ ]:
import cdsapi
import os
import geopandas as gpd
import xarray as xr
import rasterio as rio

In [ ]:
absolutepath = os.getcwd()
absolutepath 

### Set and end date:

In [ ]:
start_date = '2020-03-01'
end_date = '2020-03-20'

### Import Bounding Box:

In [ ]:
area_path = absolutepath + '/bounding_box/bounding_box_buffer20.gpkg'
area = gpd.read_file(area_path).to_crs(4326)
bounds = area.total_bounds

In [ ]:
print(bounds)

# ADS Atmosphere Copernicus
Example: https://github.com/ecmwf-projects/copernicus-training/blob/master/2021-02-Copernicus-ECMWF-data-tutorial.ipynb

Link to ADS API from (https://ads.atmosphere.copernicus.eu/api-how-to 
):

In [ ]:
ATM_ID = 'https://ads.atmosphere.copernicus.eu/api/v2'
ATM_KEY = '7829:e0d40eca-947c-4eb8-91f5-c5cfbcaa2172'

In [ ]:
c_atm = cdsapi.Client(ATM_ID, ATM_KEY)

- - -

# [CAMS - PM2.5 - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for PM2.5 over the study area:

In [ ]:
pm25_path = absolutepath+r'\temp\pm25_cams.nc'
pm25 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'particulate_matter_2.5um',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    pm25_path
    )

Read pm25 .nc file:

In [ ]:
pm25 = xr.open_dataset(pm25_path)

In [ ]:
pm25_mean = pm25.mean(dim='time')

Write pm25_mean .nc file:

In [ ]:
pm25_mean.to_netcdf(absolutepath+r'\temp\pm25_cams.nc')

- - -

# [CAMS - Ammonia - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for ammonia over the study area:

In [ ]:
nh3_path = absolutepath+r'\temp\nh3_cams.nc'
nh3 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'ammonia',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    nh3_path
    )

Read ammonia .nc file:

In [ ]:
nh3 = xr.open_dataset(nh3_path)

Calculate average PM2.5 for the selected days:

In [ ]:
nh3_mean = nh3.mean(dim='time')

Write ammonia_mean .nc file:

In [ ]:
nh3_mean.to_netcdf(absolutepath+r'\temp\nh3_cams.nc', 'w')

- - -

# [CAMS - Sulphur Dioxide - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for sulphur sioxide (so2) over the study area:

In [ ]:
so2_path = absolutepath + r'/temp/so2_cams.nc'
so2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'sulphur_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    so2_path
    )

Read sulphur dioxide .nc file:

In [ ]:
so2 = xr.open_dataset(so2_path)

Calculate average sulphure dioxide concentration for the selected days:

In [ ]:
so2_mean = so2.mean(dim='time')

Write so2_mean .nc file:

In [ ]:
so2_mean.to_netcdf(so2_path)

- - -

# [CAMS - Nitrogen Dioxide - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

In [ ]:
no2_path = absolutepath + r'/temp/no2_cams.nc'
no2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no2_path
    )

In [ ]:
no2 = xr.open_dataset(no2_path)

In [ ]:
no2_mean = no2.mean(dim='time')
no2_mean

In [ ]:
no2_mean.to_netcdf(no2_path)

- - -

# Nitrogen Monoxide

In [ ]:
no_path = absolutepath + r'/temp/no_cams.nc'
no = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no_path
    )

In [ ]:
no = xr.open_dataset(no_path)

In [ ]:
no_mean = no.mean(dim='time')
no_mean

In [ ]:
no_mean.to_netcdf(no_path)

- - -

# Carbon monoxide

In [ ]:
co_path = absolutepath + r'/temp/co_cams.nc'
co = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'carbon_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    co_path
    )

In [ ]:
co = xr.open_dataset(co_path)

In [ ]:
co_mean = co.mean(dim='time')
co_mean

In [ ]:
co_mean.to_netcdf(co_path)

---

# Dust

In [ ]:
dust_path = absolutepath + r'/temp/dust_cams.nc'
dust = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'dust',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    dust_path
    )

In [ ]:
dust = xr.open_dataset(dust_path)

In [ ]:
dust_mean = dust.mean(dim='time')
dust_mean

In [ ]:
dust_mean.to_netcdf(dust_path)

# PM10

In [ ]:
pm10_path = absolutepath + r'/temp/pm10_cams.nc'
pm10 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'particulate_matter_10um',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    pm10_path)

In [ ]:
pm10 = xr.open_dataset(pm10_path)

In [ ]:
pm10_mean = pm10.mean(dim='time')
pm10_mean

In [ ]:
pm10_mean.to_netcdf(pm10_path)

# Non Methane VOCs

In [ ]:
nmvocs_path = absolutepath + r'/temp/nmvocs_cams.nc'
nmvocs = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'non_methane_vocs',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    nmvocs_path)

In [ ]:
nmvocs = xr.open_dataset(nmvocs_path)

In [ ]:
nmvocs_mean = nmvocs.mean(dim='time')
nmvocs_mean

In [ ]:
nmvocs_mean.to_netcdf(nmvocs_path)

# CDS Climate Copernicus

Link to ADS API from (https://cds.climate.copernicus.eu/api-how-to):<br>
Example: https://github.com/ecmwf-projects/copernicus-training/blob/master/100_climate_data_store_intro.ipynb

In [ ]:
# CLIM_ID = 'https://cds.climate.copernicus.eu/api/v2'
# CLIM_KEY = '114120:ff103cbe-66d7-4fde-bc68-0bb30039dc50'

In [ ]:
# c_clim = cdsapi.Client()

# [ERA5 - Temperature 2 m level - Renalysis](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form) 

In [ ]:
# temp2m_path = absolutepath + r'/temp/temp_2m.nc'
# c_clim.retrieve(
#     'reanalysis-era5-land',
#     {
#         'format': 'netcdf',
#         'variable': '2m_temperature',
#         'year': '2021',
#         'month': '11',
#         'day': [
#             '01'
#         ],
#         'area': [
#             bounds[3], bounds[0], bounds[1],
#             bounds[2],
#         ],
#         'time': '12:00',
#     },
#     temp2m_path)

In [ ]:
# temp_2m = xr.open_dataset(temp2m_path)

In [ ]:
# temp_2m_mean = temp_2m.mean(dim='time')
# temp_2m_mean

In [ ]:
# temp_2m_mean.to_netcdf(temp2_path)

In [ ]:
# bounds